In [1]:
from bs4 import BeautifulSoup
import urllib.request
import string
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [19]:
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics.pairwise import cosine_similarity 
import numpy as np

In [25]:
import itertools

In [3]:
# scrap dari digilib

In [4]:
docs_num = 20

In [5]:
digilib_url = 'http://digilib.its.ac.id/publisher/51100/'

In [6]:
paper = []

In [7]:
for i in range(0,docs_num,20):
    page = urllib.request.urlopen(digilib_url)
    soup = BeautifulSoup(page, 'html.parser')
    docs = soup.find_all('span', attrs={'class': 'style5'})
    link = []
    for x in docs:
        try:
            link.append(x.find('a').get('href'))
        except:
            pass
    for x in tqdm(link[:20], desc='paper', unit='paper'):
        page = urllib.request.urlopen(x)
        soup = BeautifulSoup(page, 'html.parser')
        try:
            title = soup.find('h2', attrs={'class': 'isi'}).find('i').getText()
            abstract = soup.find('span', attrs={'class': 'teks'}).find('p').getText()
            paper.append([x, title, abstract])
        except:
            pass

paper: 100%|████████████████████████████████| 20/20 [00:16<00:00,  1.22paper/s]


In [8]:
len(paper)

18

In [9]:
# preprocessing

In [10]:
factory = StopWordRemoverFactory()
stopword = factory.create_stop_word_remover()
stemmer = StemmerFactory().create_stemmer()

In [11]:
words = []
processed_paper = []

In [12]:
for x in tqdm(paper, desc='paper', unit='paper'):
    text = x[2]
    text = text.lower()
    remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)
    text = text.translate(remove_punctuation_map)
    text = stopword.remove(text)
    text = text.split()
    text = [stemmer.stem(x) for x in text]
    processed_paper.append(' '.join(text))
    text = list(set(text))
    words += text

paper: 100%|████████████████████████████████| 18/18 [02:39<00:00,  8.86s/paper]


In [13]:
# generate thesaurus

In [14]:
thesaurus = {}

In [15]:
words = list(set(words))

In [94]:
for x in tqdm(words, desc='word', unit='word'):
    name = x
    data = { "q": name }
    encoded_data = urllib.parse.urlencode(data).encode("utf-8")
    content = urllib.request.urlopen("http://www.sinonimkata.com/search.php", encoded_data)
    soup = BeautifulSoup(content, 'html.parser')
    try:
        synonym = soup.find('td', attrs={'width': '90%'}).find_all('a')
        synonym = [x.getText() for x in synonym]
        thesaurus[x] = [x] + synonym
    except:
        thesaurus[x] = [name]

word: 100%|████████████████████████████████| 502/502 [11:27<00:00,  1.37s/word]


In [17]:
# tf_idf

In [20]:
vectorizer = TfidfVectorizer(use_idf=True)

In [111]:
query = 'hosting dengan aplikasi internet'

In [112]:
query = query.lower()
remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)
query = query.translate(remove_punctuation_map)
query = stopword.remove(query)
query = query.split()
query = [stemmer.stem(x) for x in query]

In [113]:
query

['hosting', 'aplikasi', 'internet']

In [122]:
product_query = []
list_synonym = [thesaurus[x] for x in query]

In [100]:
max_score = 0
max_q = ''

In [101]:
for x in itertools.product(*list_synonym):
    q = [' '.join(x)]
    paper_tfidf = vectorizer.fit_transform(q + processed_paper)
    q = paper_tfidf[0]
    result = cosine_similarity(paper_tfidf, q)
    idx = np.argsort(-result,axis=0)[1].flatten()
    if result[idx[0]][0] > max_score:
        max_score = result[idx[0]][0]
        max_q = [' '.join(x)]

In [104]:
print(max_score, max_q)

0.4001678436644695 ['hosting aplikasi internet']


In [106]:
paper_tfidf = vectorizer.fit_transform(max_q + processed_paper) 
max_q = paper_tfidf[0]
result = cosine_similarity(paper_tfidf, max_q)
idx = np.argsort(-result,axis=0)[1:6].flatten()
for x in idx:
    print('Paper', x, 'score :', result[x][0])

Paper 3 score : 0.4001678436644695
Paper 4 score : 0.4001678436644695
Paper 1 score : 0.14351678792357103
Paper 2 score : 0.14351678792357103
Paper 18 score : 0.03263376888469902


In [107]:
for x in range(len(idx)): 
    print('Result', x+1) 
    print(paper[idx[x]-1][1]) 
    print(paper[idx[x]-1][2])
    print()

Result 1
RANCANG BANGUN LAYANAN PLATFORM AS A SERVICE PAAS UNTUK MENDUKUNG SISTEM MULTI-TENANCY PENGEMBANGAN APLIKASI BERBASIS KOMPUTASI AWAN
Layanan hosting aplikasi atau sering disebut Application Hosting  Web Hosting merupakan layanan yang jamak tersedia di Indonesia. Layanan ini memberikan wadah bagi siapapun untuk menempatakan aplikasi Web mereka beserta basis datanya di jaringan Internet tanpa perlu menyediakan sendiri infrastruktur Internet terdedikasi. Layanan ini memiliki berbagai macam bentuk. Salah satu yang paling banyak tersedia di Indonesia adalah Shared Web Hosting yang menyediakan layanan yang sangat murah. Pada jenis layanan ini penyewa hanya diberikan satu buah platform yang sama dengan aplikasi yang disimpan bersamaan pada satu buah server sehingga memungkinkan adanya bottleneck. Selain itu terdapat jenis layanan lain yaitu Virtual Private Server VPS yang menyediakan kendali penuh pada sistem melalui teknik yang disebut virtualisasi namun dengan konfigurasi yang sang